"""
**PPMI Data Preprocessing - Stage 1: Raw Data Merger**

**Objective:**
Collect and merge key PPMI datasets into a unified tabular format for downstream analysis.

**Input Data:**
- MRI (FreeSurfer: cortical thickness, subcortical volumes)
- DAT-SPECT (striatal binding ratios)
- UPDRS (Part III motor scores)
- Demographics (age, sex, education)
- Genotype (APOE, SNCA, etc.)

**Processing Steps:**
1. Load raw CSV files from PPMI
2. Select clinically relevant features
3. Standardize patient IDs and visit labels (BL/SC)
4. Left-merge datasets on PATNO to retain all available data

**Output:**
- Raw merged dataset (PPMI_Merged_Raw.csv) with missing values preserved

**Next Steps (Stage 2):**
- Advanced imputation (MICE)
- Feature engineering
- Outlier handling
"""

# Task
Refactor Experimental Python script to be clean, readable, and reusable, suitable for adding to GitHub. The script should load and clean multiple datasets (demographics, DAT-SPECT, UPDRS, MRI cortical thickness, MRI subcortical volumes, and age at visit), merge them based on 'PATNO', and save the final merged dataframe. The refactored code should include functions for data cleaning and merging, a main execution block, docstrings, type hints, and comments.

## Define functions for each data cleaning step

### 1:
Create separate functions for cleaning each dataset (demographics, DAT-SPECT, UPDRS, MRI cortical thickness, MRI subcortical volumes, and age at visit).


In [3]:
import pandas as pd

def clean_demographics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the demographics DataFrame by selecting key columns and dropping duplicates.

    Args:
        df: Raw demographics DataFrame.

    Returns:
        Cleaned DataFrame with PATNO and SEX, with duplicates on PATNO removed.
    """
    demo_clean = df[['PATNO', 'SEX']].drop_duplicates('PATNO')
    return demo_clean

**Reasoning**:
Define the function to clean the DAT-SPECT data, handling date conversion, sorting, selecting baseline data, and renaming columns.



In [5]:
def clean_datscan(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the DAT-SPECT DataFrame to get baseline scan data.

    Args:
        df: Raw DAT-SPECT DataFrame.

    Returns:
        Cleaned DataFrame with baseline DAT-SPECT metrics and scan date.
    """
    # 1. Convert scan date to datetime for proper sorting
    df['DATSCAN_DATE'] = pd.to_datetime(df['DATSCAN_DATE'], format='%m/%Y')

    # 2. Sort by patient and scan date (earliest first)
    datscan_sorted = df.sort_values(['PATNO', 'DATSCAN_DATE'])

    # 3. Keep only the first scan per patient (baseline)
    baseline_dat = datscan_sorted.groupby('PATNO', as_index=False).first()

    # 4. Select and rename key columns
    cols_to_keep = {
        'PATNO': 'PATNO',
        'EVENT_ID': 'EVENT_ID',
        'DATSCAN_CAUDATE_L': 'CAUDATE_L',
        'DATSCAN_CAUDATE_R': 'CAUDATE_R',
        'DATSCAN_PUTAMEN_L': 'PUTAMEN_L',
        'DATSCAN_PUTAMEN_R': 'PUTAMEN_R',
        'DATSCAN_DATE': 'SCAN_DATE'
    }
    baseline_dat = baseline_dat[list(cols_to_keep.keys())].rename(columns=cols_to_keep)

    # 5. Standardize EVENT_ID to 'BL'
    baseline_dat['EVENT_ID'] = 'BL'

    return baseline_dat

**Reasoning**:
Define the function to clean the UPDRS data, filtering for baseline events and selecting relevant columns.



In [7]:
def clean_updrs(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the UPDRS DataFrame to get baseline motor scores.

    Args:
        df: Raw UPDRS DataFrame.

    Returns:
        Cleaned DataFrame with baseline UPDRS-III total and Hoehn & Yahr stage.
    """
    # Keep only BL records (ignore SC/Vxx visits)
    updrs_bl = df[df['EVENT_ID'] == 'BL'].copy()

    cols_to_keep = ['PATNO', 'EVENT_ID', 'NP3TOT', 'NHY']  # Motor score + Hoehn & Yahr stage
    updrs_bl = updrs_bl[cols_to_keep]

    return updrs_bl

**Reasoning**:
Define the function to clean the MRI subcortical volumes data by selecting the specified columns.



In [8]:
def clean_mri_vol(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the MRI subcortical volumes DataFrame by selecting relevant columns.

    Args:
        df: Raw MRI subcortical volumes DataFrame.

    Returns:
        Cleaned DataFrame with selected subcortical volume metrics.
    """
    vol_selected = [
        'PATNO', 'EVENT_ID',

        # Subcortical motor/cognitive structures
        'Left_Putamen', 'Right_Putamen',
        'Left_Caudate', 'Right_Caudate',
        'Left_Pallidum', 'Right_Pallidum',
        'Left_Thalamus', 'Right_Thalamus',
        'Left_Accumbens_area', 'Right_Accumbens_area',
        'Left_Hippocampus', 'Right_Hippocampus',
        'Left_Amygdala', 'Right_Amygdala',

        # Global brain volumes
        'BrainSegVol', 'BrainSegVol_to_eTIV',
        'EstimatedTotalIntraCranialVol', 'CortexVol',
        'TotalGrayVol', 'SubCortGrayVol', 'CerebralWhiteMatterVol'
    ]
    mri_vol_clean = df[vol_selected]
    return mri_vol_clean

**Reasoning**:
Define the function to clean the MRI cortical thickness data by selecting the specified columns.



In [9]:
def clean_mri_ct(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the MRI cortical thickness DataFrame by selecting relevant columns.

    Args:
        df: Raw MRI cortical thickness DataFrame.

    Returns:
        Cleaned DataFrame with selected cortical thickness metrics.
    """
    ct_selected = [
        'PATNO', 'EVENT_ID',

        # Early Braak regions
        'lh_entorhinal', 'rh_entorhinal',
        'lh_parahippocampal', 'rh_parahippocampal',

        # Motor cortex
        'lh_precentral', 'rh_precentral',
        'lh_paracentral', 'rh_paracentral',

        # Temporal & fusiform (cognitive)
        'lh_middletemporal', 'rh_middletemporal',
        'lh_inferiortemporal', 'rh_inferiortemporal',
        'lh_fusiform', 'rh_fusiform',

        # DMN-related
        'lh_posteriorcingulate', 'rh_posteriorcingulate',
        'lh_precuneus', 'rh_precuneus',

        # Frontal/limbic
        'lh_medialorbitofrontal', 'rh_medialorbitofrontal',
        'lh_rostralanteriorcingulate', 'rh_rostralanteriorcingulate',

        # Global cortical thickness
        'lh_MeanThickness', 'rh_MeanThickness'
    ]
    mri_ct_clean = df[ct_selected]
    return mri_ct_clean

**Reasoning**:
Define the function to clean the age at visit data, standardizing event IDs and selecting the baseline age for each patient.



In [10]:
def clean_age_visit(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans the age at visit DataFrame to get the baseline age per patient.

    Args:
        df: Raw age at visit DataFrame.

    Returns:
        Cleaned DataFrame with baseline age per patient.
    """
    # Standardize visit labels
    age_df = df.copy()
    age_df['EVENT_ID'] = age_df['EVENT_ID'].replace({'SC': 'BL'})  # Treat SC as BL

    # Get best available age per patient
    baseline_ages = (
        age_df[age_df['EVENT_ID'] == 'BL']
        .sort_values(['PATNO', 'AGE_AT_VISIT'])  # Break ties consistently
        .drop_duplicates('PATNO', keep='first')
        [['PATNO', 'AGE_AT_VISIT']]
        .rename(columns={'AGE_AT_VISIT': 'AGE'})
    )

    return baseline_ages

## Define a function for merging the cleaned data

### 2:
Create a function that takes the cleaned dataframes as input and performs the left merge on 'PATNO'.


**Reasoning**:
Define the `safe_merge` function and the `merge_dataframes` function as instructed to merge the cleaned dataframes.



In [11]:
from functools import reduce

def safe_merge(left: pd.DataFrame, right: pd.DataFrame) -> pd.DataFrame:
    """Performs a left merge of two DataFrames on the 'PATNO' column."""
    return pd.merge(left, right, on='PATNO', how='left')

def merge_dataframes(dataframes: list[pd.DataFrame]) -> pd.DataFrame:
    """
    Merges a list of cleaned dataframes on the 'PATNO' column.

    Ensures 'PATNO' is string type and drops duplicates by PATNO before merging.

    Args:
        dataframes: A list of pandas DataFrames to merge.

    Returns:
        The final merged DataFrame.
    """
    processed_dataframes = []
    for df in dataframes:
        # Ensure PATNO is string type and drop duplicates by PATNO
        processed_df = df.astype({'PATNO': str}).drop_duplicates('PATNO')
        processed_dataframes.append(processed_df)

    # Use reduce to perform successive left merges
    final_merged_df = reduce(safe_merge, processed_dataframes)

    return final_merged_df

## Create a main execution block

### 3:
Add a section that loads the raw data, calls the cleaning functions, calls the merging function, and saves the final merged dataframe.


**Reasoning**:
Define the main execution block, load the data, call the cleaning and merging functions, and save the result.



In [15]:
if __name__ == "__main__":
    # Load each dataset
    # Handle DtypeWarning for 'updrs' by specifying dtype or low_memory
    demographics = pd.read_csv('Demographics_10Jun2025.csv')
    datscan = pd.read_csv('DaTScan_SBR_Analysis_07Jun2025.csv')
    updrs = pd.read_csv('MDS-UPDRS_Part_III_16May2025.csv', low_memory=False)
    mri_ct = pd.read_csv('FS7_APARC_CTH_13May2025.csv')
    mri_vol = pd.read_csv('FS7_ASEG_VOL_13May2025.csv')
    age_visit = pd.read_csv('Age_at_visit_16May2025.csv')
    # 'imputed_data1' is needed for APOE and COHORT_DEFINITION
    imputed_data1 = pd.read_csv("ppmi_clean_imputed (1).csv")

    # Clean each dataset
    demo_clean_df = clean_demographics(demographics)
    datscan_clean_df = clean_datscan(datscan)
    updrs_clean_df = clean_updrs(updrs)
    mri_vol_clean_df = clean_mri_vol(mri_vol)
    mri_ct_clean_df = clean_mri_ct(mri_ct)
    age_visit_clean_df = clean_age_visit(age_visit)

    # Clean the necessary columns from imputed_data1 for merging
    imputed_data_clean_df = imputed_data1[["PATNO","APOE4","COHORT_DEFINITION"]].drop_duplicates('PATNO')


    # Create a list of cleaned dataframes
    cleaned_dataframes = [
        demo_clean_df,
        age_visit_clean_df,
        datscan_clean_df,
        mri_vol_clean_df,
        mri_ct_clean_df,
        updrs_clean_df,
        imputed_data_clean_df
    ]

    # Modify the merge_dataframes function to handle duplicate columns
    def safe_merge(left: pd.DataFrame, right: pd.DataFrame) -> pd.DataFrame:
        """Performs a left merge of two DataFrames on the 'PATNO' column, handling duplicate columns."""
        return pd.merge(left, right, on='PATNO', how='left', suffixes=('', '_drop'))

    def merge_dataframes(dataframes: list[pd.DataFrame]) -> pd.DataFrame:
        """
        Merges a list of cleaned dataframes on the 'PATNO' column.

        Ensures 'PATNO' is string type and drops duplicates by PATNO before merging.

        Args:
            dataframes: A list of pandas DataFrames to merge.

        Returns:
            The final merged DataFrame.
        """
        processed_dataframes = []
        for df in dataframes:
            # Ensure PATNO is string type and drop duplicates by PATNO
            processed_df = df.astype({'PATNO': str}).drop_duplicates('PATNO')
            processed_dataframes.append(processed_df)

        # Use reduce to perform successive left merges
        final_merged_df = reduce(safe_merge, processed_dataframes)

        # Drop duplicate columns created by the merge
        final_merged_df = final_merged_df.loc[:, ~final_merged_df.columns.str.endswith('_drop')]

        return final_merged_df


    # Merge the dataframes
    final_merged_df = merge_dataframes(cleaned_dataframes)

    # Save the final merged dataframe
    final_merged_df.to_csv('merged_ppmi_dataset.csv', index=False)

    print("Merged dataset saved to 'merged_ppmi_dataset.csv'")
    display(final_merged_df.head())

Merged dataset saved to 'merged_ppmi_dataset.csv'


,PATNO,SEX,AGE,EVENT_ID,CAUDATE_L,CAUDATE_R,PUTAMEN_L,PUTAMEN_R,SCAN_DATE,Left_Putamen,...,lh_medialorbitofrontal,rh_medialorbitofrontal,lh_rostralanteriorcingulate,rh_rostralanteriorcingulate,lh_MeanThickness,rh_MeanThickness,NP3TOT,NHY,APOE4,COHORT_DEFINITION
0,3000,0,69.1,BL,3.43,2.99,2.63,2.94,2011-01-01,4380.2,...,2.405,2.292,2.975,2.848,2.30500,2.32243,4.0,0.0,0.0,Healthy Control
1,3001,1,65.1,BL,1.92,2.00,0.65,0.80,2011-06-01,7369.2,...,2.458,2.298,2.645,2.520,2.31882,2.28570,12.0,1.0,0.0,Parkinson's Disease
2,3002,0,67.6,BL,3.72,2.92,1.78,1.01,2011-06-01,5060.8,...,2.270,2.429,2.860,2.515,2.48731,2.45438,17.0,2.0,0.0,Parkinson's Disease
3,3003,0,56.6,BL,2.54,3.63,0.68,1.11,2011-08-01,4832.5,...,2.439,2.477,2.639,2.544,2.47113,2.41209,29.0,2.0,1.0,Parkinson's Disease
4,3004,1,59.3,BL,5.30,5.09,2.97,3.54,2011-08-01,5383.9,...,2.409,2.268,2.881,2.476,2.46058,2.39909,2.0,0.0,0.0,Healthy Control
